In [ ]:
!pip install diffusers accelerate safetensors transformers gradio==3.37.0

# Manual inference

In [ ]:
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler

model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

url = "https://raw.githubusercontent.com/timothybrooks/instruct-pix2pix/main/imgs/example.jpg"
def download_image(url):
    image = PIL.Image.open(requests.get(url, stream=True).raw)
    image = PIL.ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    return image
image = download_image(url)

prompt = "turn him into cyborg"
images = pipe(prompt, image=image, num_inference_steps=10, image_guidance_scale=1).images
images[0]

# Gradio implementation

In [ ]:
import gradio as gr
from IPython import get_ipython
from IPython.display import display
import PIL
import requests
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
import os

# Load the model (same as before)
model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# Define a function for downloading and saving images
def download_and_save_image(url, filename):
    """Downloads an image and saves it to a file."""
    image = PIL.Image.open(requests.get(url, stream=True).raw)
    image = PIL.ImageOps.exif_transpose(image)
    image = image.convert("RGB")
    image.save(filename)  # Save the image to a file
    return filename # Return the filename

# Define the main inference function for the Gradio app
def inference(image, prompt):
    # No need to convert to PIL Image as it's already in PIL Image format
    # image = PIL.Image.fromarray(image)
    images = pipe(prompt, image=image, num_inference_steps=10, image_guidance_scale=1).images
    images[0].save("output.png")
    return images[0]

# Download and save example images
example_image_1 = download_and_save_image("https://raw.githubusercontent.com/timothybrooks/instruct-pix2pix/main/imgs/example.jpg", "example1.jpg")
example_image_2 = download_and_save_image("https://raw.githubusercontent.com/timothybrooks/instruct-pix2pix/main/imgs/example.jpg", "example2.jpg")

iface = gr.Interface(
    fn=inference,
    inputs=[gr.Image(type="pil"), gr.Textbox(lines=1, placeholder="Enter your prompt here...")],
    outputs=gr.Image(type="pil"),
    title="Instruct Pix2Pix Demo",
    description="Turn your images into something new with Instruct Pix2Pix!",
    examples=[
          [example_image_1, "turn him into cyborg"], # Use filenames instead of image objects
          [example_image_2, "make him a robot"], # Use filenames instead of image objects
          ]
)

iface.launch(share=True, debug=True)